In [ ]:
# | default_exp wb_veg

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export

import warnings
import collections
import numpy as np
from typing import Dict
from pandera.typing import DataFrame
from sureau_ecos_py.climate_utils import (
    day_length,
    compute_pet,
    potential_par,
    rg_units_conversion,
    compute_vpd_from_t_rh,
    calculate_rh_diurnal_pattern,
    rg_watt_ppfd_umol_conversions,
    calculate_radiation_diurnal_pattern,
    calculate_temperature_diurnal_pattern,
)
from sureau_ecos_py.create_simulation_parameters import (
    create_simulation_parameters,
)
from sureau_ecos_py.create_climate_data import create_climate_data
from sureau_ecos_py.create_modeling_options import create_modeling_options

In [ ]:
# | export



#### __Example: __